# Day 3: Learning Structured Predictors

In Day 2, we focused on generative sequence classifiers - HMMs. Today's focus is on discriminative classifiers. Recall that:

* **generative classifiers** try to model the probability distribution of the data, $P(X, Y)$;

* **discriminative classifiers** only model the conditional probability of each class given the observed data, $P(Y\,|\,X)$.

In Day 1, we implemented discriminative models for classification tasks. Today, we extend this concept to the classification of _sequential_ data.

## Summary

You will be using two discriminative classifiers to do part-of-speech tagging:
* Conditional Random Fields (CRF) and
* Structured Perceptron.

Your tasks for this lab session are:

* to train a CRF model using two different sets of features (exercises 3.1 and 3.2); 
* to implement the structured perceptron algorithm (exercise 3.3); 
* to compare the performance of the Structured Perceptron with that of CRFs (exercise 3.4).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
# We will this append to ensure we can import lxmls toolking
sys.path.append('../lxmls-toolkit')

## Introduction

Discriminative sequence models aim to solve the following:

$$\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ P(Y=y\,|\,X=x)=\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ \boldsymbol{w}\cdot\boldsymbol{f}(x, y)$$

where $\boldsymbol{w}$ is the model's weight vector, and $\boldsymbol{f}(x, y)$ is a feature vector. Notice that now both $y$ and $x$ are $N$-dimensional vectors, whereas in Day 1, these variables were just scalar numbers.

In Day 2, sequences were scored using the log-probability. On today's models we are still scoring the sequences; the only difference is the scores are now computed as the product of the weights with the feature vector:


| score | Hidden Markov Models (Day 2) | Discriminative Models (Today) |
| ------------------------------- | ---------------- | ---------------- |
| $\textrm{score}_\textrm{emiss}$ | $\log P(x_i\,|\,y_i) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i)$ |
| $\textrm{score}_\textrm{init}$ | $\log P(y_1\,|\,\mathrm{start}) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1)$ |
| $\textrm{score}_\textrm{trans}$ | $\log P(y_{i+1}\,|\,y_i) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1})$ |
| $\textrm{score}_\textrm{final}$ | $\log P(\mathrm{stop}\,|\,y_N) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N)$ |

Notice that the scores computed using the feature vector depend on two sequential values of the output variable, $y$, but may depend on the whole observated input, $x$. We can now rewrite the above expression as

$$
\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ 
\sum_{i=1}^N \boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1}) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N) = 
\\
\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ 
\sum_{i=1}^N \textrm{score}_\textrm{emiss}(i, x, y_i) + 
\textrm{score}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\textrm{score}_\textrm{trans}(i, x, y_i, y_{i+1}) +
\textrm{score}_\textrm{final}(x, y_N)
$$

The advantage of these linear models over HMMs is that the feature vector may contain more general features. [TODO example of what these 'standard' and 'extended' features could be]

### Decoding

One important thing to notice is that the decoding process - the process by which we pick the most likely label $y_i$ for the observation $x_i$ - stays the same. This means *we do not need to develop new decoders,* only new functions to compute the scores. Because of this, we will keep using the Viterbi and Forward-Backward algorithms developed on Day 2.

### Training the classifier

Today we will cover two different approaches to training sequential discriminative models. Given a training set with $M$ observation-label pairs, $\{(x_m, y_m)\}_{m=1}^M$ (note that $x_m$, $y_m$ are $N$-dimensional vectors, as $m$ indexes the training sample):

* **Conditional Random Fields** maximize the log-likelihood over $w$ on the observed training set.
* **Structured Perceptron** iteratively updates $w$ in order to correctly classify the training set.

## Conditional Random Fields

CRFs are the generalization of the Maximum Entropy classifier for sequences. The general concept is the same, with a couple of diferences to be discussed below. They are trained by solving the following optimization problem:

$$
\hat{w} = \underset{w}{\textrm{arg max}}\ \sum_{m=1}^M\log P_w(Y=y_m\,|\,X=x_m)
$$

where [TODO fix parenthesis... apparently \left( doesnt work in notebooks]

$$
P_w(Y=y_m\,|\,X=x_m) =
\frac{1}{Z(w, x)}\ \exp (\sum_{i=1}^N \boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1}) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N))\\
Z(w, x) = \sum_{y\,\in\,\Lambda^N} \exp (\sum_{i=1}^N \boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1}) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N))
$$

As before, the partition function $Z(w,x)$ ensures the sum of probabilities over all possible labels $y\in\Lambda^N$ is equal to 1.

To avoid overfitting, it is common to add the Euclidean norm function as a regularization term. This is equivalent
to considering a zero-mean Gaussian prior on the weight vector. The optimization problem becomes:

$$
\hat{w} = \underset{w}{\textrm{arg max}}\ \sum_{m=1}^M\log P_w(Y=y_m\,|\,X=x_m) - \frac{\lambda}{2}||w||^2
$$

which is precisely the structured variant of the maximum entropy method discussed on Day 1. Unlike with HMMs, the above problem has to be solved numerically.

### Differences with respect to ME algorithm

* CRe does not compute posterior marginals, $P(Y=y\,|\,X=x)$ for every possible $y\in\Lambda^N$, as there are exponentially many possible $y$'s. Instead, it decomposes the model into parts — nodes and edges — and computes the posteriors for those parts, that is, $P(Y_i=y_i\,|\,X=x)$ and $P(Y_i=y_i, Y_{i+1}=y_{i+1}\,|\,X=x)$. The crucial point is that these quantities can be computed using the forward-backward algorithm.

* Instead of updating the features for all possible outputs $y′ ∈ \Lambda^N$, we again exploit the decomposition into parts above and update only “local features” at the nodes and edges. [TODO clarify this]

### Pseudo Code

Below is pseudo code to optimize a CRF with the stochastic gradient descent (SGD) algorithm. Our toolkit also includes an implementation of a quasi-Newton method, L-BFGS, which converges faster. For the purpose of this exercise, however, we will stick with SGD.

[TODO pseudo code?]

## Exercises
​

Objectives:


* train a CRF using different feature sets for part-of-speech tagging;

* evaluate the model on the training, development and test sets.


Files used:

* class CRFOnline in lxmls/sequences/crf_online.py file

* class PostagCorpus in lxmls/sequences/readers/pos_corpus.py file

* class IDFeatures in lxmls/sequences/id_feature.py file

* class ExtendedFeatures in lxmls/sequences/extended_feature.py file

# About Feature Generation

Given a dataset,

in order to build the features

- An instance from IDFeatures (we will call it feature_mapper) must be instanciated
- feature_mapper.build_features() must be executed



In [2]:
import lxmls
import lxmls.sequences.crf_online as crfo
import lxmls.readers.pos_corpus as pcc
import lxmls.sequences.id_feature as idfc
import lxmls.sequences.extended_feature as exfc

from lxmls.readers import pos_corpus
corpus = lxmls.readers.pos_corpus.PostagCorpus()
data_path = "../lxmls-toolkit/data/"

train_seq = corpus.read_sequence_list_conll(data_path + "/train-02-21.conll", 
                                            max_sent_len=10, max_nr_sent=1000)


In [3]:
print ("There are", len(train_seq), "samples in train_seq")

There are 1000 samples in train_seq


In [4]:
train_seq[0]

Ms./noun Haag/noun plays/verb Elianti/noun ./. 

In [5]:
feature_mapper= lxmls.sequences.id_feature.IDFeatures(train_seq)

In [6]:
feature_mapper.feature_list

[]

Now we will call ```feature_mapper.build_features()``` to get the features for each training sample

In [7]:
feature_mapper.build_features()

In [8]:
print ("there are", len(feature_mapper.feature_list), "samples in the features build from train_seq")

there are 1000 samples in the features build from train_seq


In [9]:
len(feature_mapper.feature_list[0])

4

In [10]:
feature_mapper.feature_list[0]

[[[0]], [[3], [5], [7], [9]], [[10]], [[1], [2], [4], [6], [8]]]

In [11]:
feature_mapper.dataset[0]

Ms./noun Haag/noun plays/verb Elianti/noun ./. 

In [12]:
print ("\nInitial features:",     feature_mapper.feature_list[0][0])
print ("\nTransition features:",  feature_mapper.feature_list[0][1])
print ("\nFinal features:",       feature_mapper.feature_list[0][2])
print ("\nEmission features:",    feature_mapper.feature_list[0][3])


Initial features: [[0]]

Transition features: [[3], [5], [7], [9]]

Final features: [[10]]

Emission features: [[1], [2], [4], [6], [8]]


### Codification of the features

All features are saved in ``feature_mapper.feature_dict`` this represents our feature vector. If it is our feature vector why it's not a vector? Good point! In order to make the algorithm fast, the code is written using dicts, so if we access only a few positions from the dict and compute substractions it will be much faster than computing the substraction of two huge weight vectors.

Features are identifyed by **init_tag:**, **prev_tag:**,  **final_prev_tag:**, **id:**

- **init_tag:** when they are Initial features
    - Example: **``init_tag:noun``** is an initial feature that describes that the first word is a noun
    
    
- **prev_tag:** when they are transition features
    - Example: **``prev_tag:noun::noun``** is an transition feature that describes that the previous word was
      a noun and the current word is a noun.
    - Example: **``prev_tag:noun:.``** is an transition feature that describes that the previous word was
      a noun and the current word is a `.` (this is usually foud as the last transition feature since most phrases will end up with a dot)
      


- **final_prev_tag:** when they are final features
    - Example: **``final_prev_tag:.``** is a final feature stating that the last "word" in the sentence was a dot.


- **id:** when they are emission features
    - Example: **``id:plays::verb``** is an emission feature, describing that the current word is plays and the current hidden state is a verb.
    - Example: **``id:Feb.::noun``** is an emission feature, describing that the current word is "Feb." and the current hidden state is a noun.




In [13]:
inv_feature_dict = {word: pos for pos, word in feature_mapper.feature_dict.items()}

In [14]:
[inv_feature_dict[x[0]] for x in feature_mapper.feature_list[0][0]]

['init_tag:noun']

In [15]:
[inv_feature_dict[x[0]] for x in feature_mapper.feature_list[0][1]]

['prev_tag:noun::noun',
 'prev_tag:noun::verb',
 'prev_tag:verb::noun',
 'prev_tag:noun::.']

In [16]:
[inv_feature_dict[x[0]] for x in feature_mapper.feature_list[0][2]]

['final_prev_tag:.']

In [17]:
[inv_feature_dict[x[0]] for x in feature_mapper.feature_list[0][3]]

['id:Ms.::noun',
 'id:Haag::noun',
 'id:plays::verb',
 'id:Elianti::noun',
 'id:.::.']

In [18]:
print ("\nInitial features:",     feature_mapper.feature_list[1][0])
print ("\nTransition features:",  feature_mapper.feature_list[1][1])
print ("\nFinal features:",       feature_mapper.feature_list[1][2])
print ("\nEmission features:",    feature_mapper.feature_list[1][3])


Initial features: [[11]]

Transition features: [[14], [16], [5], [19], [21], [16], [24], [25]]

Final features: [[10]]

Emission features: [[12], [13], [15], [17], [18], [20], [22], [23], [8]]


In [19]:
feature_mapper.feature_list[1]

[[[11]],
 [[14], [16], [5], [19], [21], [16], [24], [25]],
 [[10]],
 [[12], [13], [15], [17], [18], [20], [22], [23], [8]]]

In [20]:
[inv_feature_dict[x[0]] for x in feature_mapper.feature_list[1][0]]

['init_tag:det']

In [21]:
[inv_feature_dict[x[0]] for x in feature_mapper.feature_list[1][1]]

['prev_tag:det::adj',
 'prev_tag:adj::noun',
 'prev_tag:noun::verb',
 'prev_tag:verb::verb',
 'prev_tag:verb::adj',
 'prev_tag:adj::noun',
 'prev_tag:noun::num',
 'prev_tag:num::.']

In [22]:
[inv_feature_dict[x[0]] for x in feature_mapper.feature_list[1][2]]

['final_prev_tag:.']

In [23]:
[inv_feature_dict[x[0]] for x in feature_mapper.feature_list[1][3]]

['id:The::det',
 'id:new::adj',
 'id:rate::noun',
 'id:will::verb',
 'id:be::verb',
 'id:payable::adj',
 'id:Feb.::noun',
 'id:15::num',
 'id:.::.']

In [24]:
len(feature_mapper.feature_dict)

2683

In [25]:
feature_mapper.feature_dict["id:Each::det"]

946

In [26]:
inv_feature_dict [11]

'init_tag:det'

In [27]:
pos = [inv_feature_dict[x]=="id:Each::det"  for x in range(len(inv_feature_dict))]

In [29]:
for x in range(len(inv_feature_dict)):
    if inv_feature_dict[x]=="id:Each::det":
        print (x, inv_feature_dict[x])

946 id:Each::det


In [30]:
inv_feature_dict

{0: 'init_tag:noun',
 1: 'id:Ms.::noun',
 2: 'id:Haag::noun',
 3: 'prev_tag:noun::noun',
 4: 'id:plays::verb',
 5: 'prev_tag:noun::verb',
 6: 'id:Elianti::noun',
 7: 'prev_tag:verb::noun',
 8: 'id:.::.',
 9: 'prev_tag:noun::.',
 10: 'final_prev_tag:.',
 11: 'init_tag:det',
 12: 'id:The::det',
 13: 'id:new::adj',
 14: 'prev_tag:det::adj',
 15: 'id:rate::noun',
 16: 'prev_tag:adj::noun',
 17: 'id:will::verb',
 18: 'id:be::verb',
 19: 'prev_tag:verb::verb',
 20: 'id:payable::adj',
 21: 'prev_tag:verb::adj',
 22: 'id:Feb.::noun',
 23: 'id:15::num',
 24: 'prev_tag:noun::num',
 25: 'prev_tag:num::.',
 26: 'id:A::det',
 27: 'id:record::noun',
 28: 'prev_tag:det::noun',
 29: 'id:date::noun',
 30: 'id:has::verb',
 31: "id:n't::adv",
 32: 'prev_tag:verb::adv',
 33: 'id:been::verb',
 34: 'prev_tag:adv::verb',
 35: 'id:set::verb',
 36: 'prev_tag:verb::.',
 37: 'init_tag:adv',
 38: 'id:Not::adv',
 39: 'id:all::det',
 40: 'prev_tag:adv::det',
 41: 'id:those::det',
 42: 'prev_tag:det::det',
 43: 'id:



### Exercise 3.1 - Basic feature set

_Start by training the model. You will receive feedback when each epoch is finished. Note that running the 20 epochs might take a while._

In [31]:
import lxmls.sequences
import lxmls.sequences.crf_online as crfo
import lxmls.readers.pos_corpus as pcc
import lxmls.sequences.id_feature as idfc
import lxmls.sequences.extended_feature as exfc

In [32]:
data_path = "../lxmls-toolkit/data/"

In [34]:
# Load the corpus
corpus = pcc.PostagCorpus()

# Load the training, test and development sequences
train_seq = corpus.read_sequence_list_conll(data_path + "/train-02-21.conll", 
                                            max_sent_len=10, max_nr_sent=1000)
test_seq = corpus.read_sequence_list_conll(data_path + "/test-23.conll",
                                           max_sent_len=10, max_nr_sent=1000)
dev_seq = corpus.read_sequence_list_conll(data_path + "/dev-22.conll", 
                                          max_sent_len=10, max_nr_sent=1000)


In [35]:
train_seq[0]

Ms./noun Haag/noun plays/verb Elianti/noun ./. 

In [36]:
type(train_seq[0])

lxmls.sequences.sequence.Sequence

In [37]:
train_seq[0].x

[42, 40, 43, 44, 41]

In [38]:
train_seq[0].y

[0, 0, 6, 0, 4]

In [39]:
# Build features
feature_mapper = lxmls.sequences.id_feature.IDFeatures(train_seq)
feature_mapper.build_features()

In [43]:
# Build features
feature_mapper = idfc.IDFeatures(train_seq)
feature_mapper.build_features()

# Train the model
# You will receive feedback when each epoch is finished.
# Note that running the 20 epochs might take a while.
crf_online = crfo.CRFOnline(corpus.word_dict, corpus.tag_dict, feature_mapper)
crf_online.num_epochs = 20
crf_online.train_supervised(train_seq)

Epoch: 0 Objective value: -5.779018
Epoch: 1 Objective value: -3.192724
Epoch: 2 Objective value: -2.717537
Epoch: 3 Objective value: -2.436614
Epoch: 4 Objective value: -2.240491
Epoch: 5 Objective value: -2.091833
Epoch: 6 Objective value: -1.973353
Epoch: 7 Objective value: -1.875643
Epoch: 8 Objective value: -1.793034
Epoch: 9 Objective value: -1.721857
Epoch: 10 Objective value: -1.659605
Epoch: 11 Objective value: -1.604499
Epoch: 12 Objective value: -1.555229
Epoch: 13 Objective value: -1.510806
Epoch: 14 Objective value: -1.470468
Epoch: 15 Objective value: -1.433612
Epoch: 16 Objective value: -1.399759
Epoch: 17 Objective value: -1.368518
Epoch: 18 Objective value: -1.339566
Epoch: 19 Objective value: -1.312636


You will receive feedback when each epoch is finished, note that running the 20 epochs might take a while. You whould get the following list:

    Epoch: 0 Objective value: -5.779018
    Epoch: 1 Objective value: -3.192724
    Epoch: 2 Objective value: -2.717537
    Epoch: 3 Objective value: -2.436614
    Epoch: 4 Objective value: -2.240491
    Epoch: 5 Objective value: -2.091833
    Epoch: 6 Objective value: -1.973353
    Epoch: 7 Objective value: -1.875643
    Epoch: 8 Objective value: -1.793034
    Epoch: 9 Objective value: -1.721857
    Epoch: 10 Objective value: -1.659605
    Epoch: 11 Objective value: -1.604499
    Epoch: 12 Objective value: -1.555229
    Epoch: 13 Objective value: -1.510806
    Epoch: 14 Objective value: -1.470468
    Epoch: 15 Objective value: -1.433612
    Epoch: 16 Objective value: -1.399759
    Epoch: 17 Objective value: -1.368518
    Epoch: 18 Objective value: -1.339566
    Epoch: 19 Objective value: -1.312636


After training is done, evaluate the learned model on the training, development and test sets.


In [44]:
# Make predictions for the various sequences using the trained model.
pred_train = crf_online.viterbi_decode_corpus(train_seq)
pred_dev = crf_online.viterbi_decode_corpus(dev_seq)
pred_test = crf_online.viterbi_decode_corpus(test_seq)

# Evaluate and print accuracies
eval_train = crf_online.evaluate_corpus(train_seq, pred_train)
eval_dev = crf_online.evaluate_corpus(dev_seq, pred_dev)
eval_test = crf_online.evaluate_corpus(test_seq, pred_test)
print ("CRF - ID Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%(eval_train,eval_dev, eval_test))

CRF - ID Features Accuracy Train: 0.949 Dev: 0.846 Test: 0.858


_Your output should be similar to this:_

_CRF - ID Features Accuracy Train: 0.949 Dev: 0.846 Test: 0.858_

_Compare with the results achieved with the HMM model (0.837 on the test set). Even when using a similar feature set, a CRF yields better results than the HMM from the previous lecture._

_Perform some error analysis and figure out what are the main errors the tagger is making. Compare them with the errors made by the HMM model._

_**Hint:** use the methods developed in the previous lecture to help you with the error analysis._

### Exercise 3.2 - Extended feature set

#### Begin Ex 3.2 ----------------------------------------------------------------------------

_Train the model again, this time using the extended feature set._

In [47]:
# Build features
feature_mapper = exfc.ExtendedFeatures(train_seq)
feature_mapper.build_features()

# Train the model
# You will receive feedback when each epoch is finished.
# Note that running the 20 epochs might take a while.
crf_online = crfo.CRFOnline(corpus.word_dict, corpus.tag_dict, feature_mapper)
crf_online.num_epochs = 20
crf_online.train_supervised(train_seq)

Epoch: 0 Objective value: -7.141596
Epoch: 1 Objective value: -1.807511
Epoch: 2 Objective value: -1.218877
Epoch: 3 Objective value: -0.955739
Epoch: 4 Objective value: -0.807821
Epoch: 5 Objective value: -0.712858
Epoch: 6 Objective value: -0.647382
Epoch: 7 Objective value: -0.599442
Epoch: 8 Objective value: -0.562584
Epoch: 9 Objective value: -0.533411
Epoch: 10 Objective value: -0.509885
Epoch: 11 Objective value: -0.490548
Epoch: 12 Objective value: -0.474318
Epoch: 13 Objective value: -0.460438
Epoch: 14 Objective value: -0.448389
Epoch: 15 Objective value: -0.437800
Epoch: 16 Objective value: -0.428402
Epoch: 17 Objective value: -0.419990
Epoch: 18 Objective value: -0.412406
Epoch: 19 Objective value: -0.405524


_And compute its accuracy._

In [48]:
# Make predictions for the various sequences using the trained model.
pred_train = crf_online.viterbi_decode_corpus(train_seq)
pred_dev = crf_online.viterbi_decode_corpus(dev_seq)
pred_test = crf_online.viterbi_decode_corpus(test_seq)

# Evaluate and print accuracies
eval_train = crf_online.evaluate_corpus(train_seq, pred_train)
eval_dev = crf_online.evaluate_corpus(dev_seq, pred_dev)
eval_test = crf_online.evaluate_corpus(test_seq, pred_test)
print ("CRF - ID Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%(eval_train,eval_dev, eval_test))

CRF - ID Features Accuracy Train: 0.984 Dev: 0.899 Test: 0.894


_Your output should be similar to this:_

_CRF - Extended Features Accuracy Train: 0.984 Dev: 0.899 Test: 0.894_

_Compare the errors obtained with the two different feature sets. Do some error analysis: what errors were correct by using more features? Can you think of other features to use to solve the errors you found?_

#### End Ex 3.2   ----------------------------------------------------------------------------




**The main lesson from this exercise is that, if you are not satisfied by the accuracy of your algorithm, you can perform some error analysis and find out which errors your algorithm is making. You can then add more features which attempt to improve those specific errors — this is known as feature engineering.**

Adding engineered features can lead to two problems:
* More features will make training and decoding more expensive. For example, if you add features that depend on the current word and the previous word, the number of new features is the square of the number of different words, which is quite large. For example, the Penn Treebank has around 40000 different words, so you are adding a lot of new features, even though not all pairs of words will ever occur. Features that depend on three words (previous, current, and next) are even more numerous.

* If features are very specific, such as the (previous word, current word, next word) one just mentioned, they might occur very rarely in the training set, which leads to overfit problems. Some of these problems (not all) can be mitigated with techniques such as smoothing, which you already learned about.

## Structured Perceptron

### Exercise 3.3 - Algorithm implementation

### Exercise 3.4 - POS Tagging